In [1]:
#|export
"""
Run simply with
$ pytest
"""

import os
import subprocess
import numpy as np
import torch
import struct

from model import ResNetMnist
from export import export_modelq8

def test_runfiles():
    """ test run.c and runq.c works with an acceptable tolerance """

    file_path = "test/data/MNIST/sorted/7/1012"
    
    # run vanilla model in test mode
    command = ["./run", "model.bin", file_path]
    with open('err.txt', mode='w') as fe:
        with open('stdout.txt', mode='w') as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res = np.loadtxt("stdout.txt")

    # run quantized model test with group size = 1 in test mode
    export_modelq8(file_path="modelq8_1.bin", gs=1)
    command = ["./runq", "modelq8_1.bin", file_path]
    with open('err1.txt', mode='w') as fe:
        with open('stdout1.txt', mode='w') as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res1 = np.loadtxt("stdout1.txt")

    # run quantized model test with group size = 1 in test mode
    export_modelq8(file_path="modelq8_2.bin", gs=2)
    command = ["./runq", "modelq8_2.bin", file_path]
    with open('err2.txt', mode='w') as fe:
        with open('stdout2.txt', mode='w') as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res2 = np.loadtxt("stdout2.txt")

    # retrieve reference values using Python model
    model = torch.load("model.pt")
    with open(file_path, "rb") as f:
        image = torch.tensor(struct.unpack('B'*(28*28), f.read(28*28))).view(1,28,28)
        image = ((image/255 - 0.5) / 0.5)
        ref = model(image).detach()
        ref = torch.nn.functional.softmax(ref, dim=1).view(-1).numpy() # python model output

    assert np.allclose(res, ref, atol=1e-5, rtol=0), "run.c: Probabilities are not close."
    assert np.allclose(res1, ref, atol=1e-5, rtol=0), "runq.c (group size = 1): Probabilities are not close."
    assert np.allclose(res2, ref, atol=1e-2, rtol=0), "runq.c (group size = 2):Probabilities are not close."


In [2]:
test_runfiles()

AttributeError: 'ResNet' object has no attribute 'dim'

In [200]:
def print_output(module, input, output):
      print(module)
      print(output.shape)
      print(output[0][0])

def test_runfiles():
    """ test run.c and runq.c works with an acceptable tolerance """

    file_path = "test/data/MNIST/sorted/7/1012"
    
    # run vanilla model in test mode
    command = ["./run", "model.bin", file_path]
    with open('err.txt', mode='w') as fe:
        with open('stdout.txt', mode='w') as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res = np.loadtxt("stdout.txt")
     # retrieve reference values using Python model
    model = torch.load("modelMNIST.pt")
    
    hook = model.group1.group[0].block.register_forward_hook(print_output)
    #hook = model.group1.register_forward_hook(print_output)
    #hook = model.bn1.register_forward_hook(print_output)
    with open(file_path, "rb") as f:
        image = torch.tensor(struct.unpack('B'*(28*28), f.read(28*28))).view(1,1,28,28)
        image = ((image/255 - 0.5) / 0.5)
        ref = model(image).detach().view(-1).numpy()
        print(ref)
    hook.remove()

    assert np.allclose(res, ref, atol=1e-5, rtol=0), "run.c: Probabilities are not close."

In [201]:
test_runfiles()

Sequential(
  (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
torch.Size([1, 64, 14, 14])
tensor([[ 1.6846e-02,  2.0052e-01,  3.4346e-01,  5.0054e-01,  5.2773e-01,
          4.9271e-01,  4.0332e-01,  3.0429e-01,  2.8427e-01,  3.3987e-01,
          4.1979e-01,  4.7369e-01,  4.9600e-01,  1.8091e-01],
        [ 1.4057e-01,  4.6611e-01,  5.9975e-01,  6.5954e-01,  6.0233e-01,
          4.0344e-01,  2.1860e-01,  1.7428e-01,  3.3946e-01,  5.2218e-01,
          6.3623e-01,  7.3134e-01,  5.9431e-01,  3.5015e-01],
        [ 4.1412e-02,  1.6437e-01,  2.0722e-01,  1.2923e-01, -2.7509e-01,
         -9.5038e-01, -1.5550e+00, -1.7116e+00, -1.0098e+00, -1.3439e-01,
          1.8560e-01,  2.4177e-0

AssertionError: run.c: Probabilities are not close.

In [39]:
model = torch.load("modelMNIST.pt")
def print_output(module, input, output):
    print(module, output)

model.maxpool.register_forward_hook(print_output)